Perry Wasdin
05/31/24

In [13]:
import pandas as pd
import numpy as np
import os

### Concatenating and filtering LIBRA-seq outputs from published datasets

In [14]:
raw_data_dir = './raw_data/'
raw_files = [i for i in os.listdir(raw_data_dir) if ('.csv' in i) or ('.xlsx' in i)]
raw_files

['200301_1655331_hiv_flu_N26_epmappingpaper_AS.xlsx',
 '200309_150517_hiv_flu_N27_epmappingpaper_AS.xlsx',
 '200324_4647-1_cov_naivedonor_AS.xlsx',
 '200324_4647-2_cov_sarsdonor_crmpaper_AS.xlsx',
 '201022_5317_sars2_potencypaper_AS.xlsx',
 '201022_5318-1_sars2_potencypaper_AS.xlsx',
 '201022_5318-2_sars2_potencypaper_AS.xlsx',
 '201113_4591-1_hiv_epmappingmaper_RV.xlsx',
 '210316_5885_sars1sars2_doubleblocker_potencypaper_AS.xlsx',
 '210616_6420_hiv_epmappingpaper_LW.xlsx',
 '210619_6421-1_hiv_epmapping_LW.xlsx',
 '210629_6421-2_hiv_epmapping_LW.xlsx',
 '211114_7128-1_cov_pooledpeds_SW.csv',
 '211119_7128-2_cov_pooledpeds_SW.csv',
 'concat_5404_paired_chains_24-03-06.csv',
 'KK_5812_concat_LIBRA-seq_outputs_PW_24-02-13.csv']

In [15]:
hiv_datasets = ['1655331', '150517', '4591-1', '6420', '6421-1', '6421-2']

In [16]:
# Get binders, returning DF with barcode, antigen name, LSS score, and UMI counts
def get_binders(lss_col, lss_thresh, umi_thresh):
    antigen = lss_col.split('.LSS')[0]
    umi_col = antigen
    if umi_col not in df.columns:
        umi_col = umi_col + '.UMI'

    binder_df = df[[lss_col, umi_col]]

    binder_df = binder_df[binder_df[lss_col] >= lss_thresh] 
    binder_df = binder_df[binder_df[umi_col] >= umi_thresh] 

    # binder_df.index = antigen + ':' + binder_df.index
    binder_df.columns = ['LSS', 'UMI']
    binder_df['Antigen'] = antigen

    return binder_df

In [17]:
meta_cols = ['SEQUENCE_INPUT.H', 'V_CALL.H', 'D_CALL.H', 'J_CALL.H',
       'JUNCTION.H', 'V_IDENTITY.H', 'J_IDENTITY.H', 'FWR1_IMGT.H', 'FWR2_IMGT.H', 'FWR3_IMGT.H',
       'FWR4_IMGT.H', 'CDR1_IMGT.H', 'CDR2_IMGT.H', 'CDR3_IMGT.H', 'C_CALL.H',
       'SEQUENCE_INPUT.L', 'V_CALL.L', 'J_CALL.L', 'SEQUENCE_VDJ.L',
       'JUNCTION.L', 'V_IDENTITY.L', 'J_IDENTITY.L', 'FWR1_IMGT.L',
       'FWR2_IMGT.L', 'FWR3_IMGT.L', 'FWR4_IMGT.L', 'CDR1_IMGT.L',
       'CDR2_IMGT.L', 'CDR3_IMGT.L', 'C_CALL.L', 'D_CALL.L', 'N']

In [18]:
binding_dfs = {}
all_data = {}

for i in raw_files:

    # Read in file from either CSV or Excel spreadsheet
    if ('.csv' in i):
        df = pd.read_csv(raw_data_dir + i, index_col='BARCODE')

    elif ('.xlsx' in i):
        df = pd.read_excel(raw_data_dir + i, index_col='BARCODE')

    id = i.split('_')[1].split('_')[0] # Get VANTAGE ID

    # Check if IgG in sample
    if not ('IGHG1' in df['C_CALL.H'].unique()):
        print('no IgG for ' + id)
        continue

    # Select isotypes to remove
    # df = df[~df['C_CALL.H'].isin(['IGHM', 'IGHD'])]
    
    df[df['N'] == 1] # Only keep cells with 1 heavy chain

    if not (len(df) > 5): # Skip samples with very low cell counts
        print('low cell count for ' + id)
        continue

    df.index = df.index + '_' + np.array(range(len(df))).astype(str)

    lss_names = [i for i in df.columns if '.LSS' in i]

    # Get binding cells with function defined above, based on LSS and UMI thresholds
    binders = pd.concat([get_binders(col, 1, 20) for col in lss_names])

    print (str(len(binders)) + ' binders for Sample ' + str(id))

    binding_dfs[id] = binders
    all_data[id] = df

215 binders for Sample 1655331
2680 binders for Sample 150517
1773 binders for Sample 4647-1
2313 binders for Sample 4647-2
449 binders for Sample 5317
357 binders for Sample 5318-1
579 binders for Sample 5318-2
2937 binders for Sample 4591-1
1814 binders for Sample 5885
359 binders for Sample 6420
56 binders for Sample 6421-1
no IgG for 6421-2
0 binders for Sample 7128-1
3015 binders for Sample 7128-2
5144 binders for Sample 5404
4305 binders for Sample 5812


In [21]:
concat_df = pd.concat(binding_dfs)

In [23]:
binding_groups = {'HIV':['HIV_ZM197', 'HIV_CZA97', 'HIV_A244', 'HIV_ConC', 'HIV_CNE55', 'BG505_6R_N332', 'CZA97_6R_N332', 'BG505sc_N332T', 'CZA97sc_N332T', 'BG505sc_N160K', 'CZA97sc_N160K', 
                         'BG505sc_K169E', 'CZA97sc_K169E', 'BG505sc_D279K', 'CZA97sc_D279K', 'BG505sc_D368R', 'CZA97sc_D368R', 'BG505sc_DKO', 'CZA97sc_DKO', 'HIV_BG505_D279K', 'HIV_BG505_D368R', 
                         'HIV_BG505_K169E', 'HIV_BG505_N160K', 'HIV_BG505_N332T', 'HIV_BG505_DKO', 'HIV_BG505_T332N'],

                'CoV':['COV_HKU1', 'COV_MERS', 'COV_SARS', 'COV_S1Fd', 'COV_OC43', 'COV_SARS2', 'CoV_HP6', 'CoV_HP1', 'CoV_SARS2', 'CoV_SARS1', 'SARS2', 'SARS1', 'MERS', 'OC43', '229E', 'SARS2_SA', 'SARS2_UK', 
                'RaTG13', 'Civet_007_2004', 'CoV_MERS', 'CoV_OC43', 'CoV_NL63', 'CoV_229E', 'CoV_SARS2D614G', 'CoV_SARS2NTD', 'CoV_SARS1RBD', 'CoV_MERSRBD', 'CoV_HKU1'],

                'Flu':['FLU_HA_H9', 'FLU_HA_H10', 'FLU_HA_Indo', 'FLU_HA_Anhui', 'FLU_HA_Michigan', 'FLU_HANC99', 'HA_NC99', 'FLU_HA_NC99'],

                'HCV': ['JFH1_E2']}

In [24]:
all_antigens = np.concatenate(list(binding_groups.values()))
concat_df = concat_df[concat_df['Antigen'].isin(all_antigens)]

In [25]:
def find_keys_by_value_in_list(my_dict, search_value):
    return [key for key, value_list in my_dict.items() if search_value in value_list][0]

def flag_polyreactive(antigen_list):
    bound_groups = [find_keys_by_value_in_list(binding_groups, ant) for ant in antigen_list]
 
    if len(np.unique(bound_groups)) > 1:
        return 'Polyreactive'
    else:
        return 'Specific'
    
def get_group(antigen):
    bound_groups = [find_keys_by_value_in_list(binding_groups, ant) for ant in [antigen]]
    return bound_groups[0]

antigens_bound = concat_df.groupby(concat_df.index)['Antigen'].unique()
antigens_bound['Reactivity'] = antigens_bound.apply(flag_polyreactive)

In [26]:
concat_df['Group'] = concat_df['Antigen'].apply(get_group)
concat_df['Group'].value_counts()

CoV    16856
HIV     6408
Flu     1233
HCV       14
Name: Group, dtype: int64

Most of the time we remove polyreactive cells

In [27]:
concat_df['Reactivity'] = antigens_bound['Reactivity']
concat_df['Reactivity'].value_counts()

Specific        22418
Polyreactive     2093
Name: Reactivity, dtype: int64

In [28]:
concat_df = concat_df[concat_df['Reactivity'] != 'Polyreactive']

In [29]:
concat_df['Vantage_ID'] = concat_df.index.get_level_values(0)

In [30]:
# For non-HIV datasets, we assume HIV binding antibodies are false positives (or non-specific binding)
def label_neg_control(row):
    if (row['Antigen'] in (binding_groups['HIV'])) and (row['Vantage_ID'] not in (hiv_datasets)):
        return 'neg control'
    else:
        return 'binding'

In [31]:
concat_df['neg_control'] = concat_df.apply(label_neg_control, axis=1)
concat_df['neg_control'].value_counts()

binding        21674
neg control      744
Name: neg_control, dtype: int64

In [32]:
concat_df = concat_df[concat_df['neg_control'] != 'neg control']

In [33]:
meta_data_concat = pd.concat(all_data).loc[concat_df.index]

# Some of the sequences are missing the raw input sequences. For purposes of getting the variable chains, we can just infill
# these with teh variable sequences assigned by IMGT

meta_data_concat['SEQUENCE_INPUT.H'] = meta_data_concat['SEQUENCE_INPUT.H'].fillna(meta_data_concat['SEQUENCE_VDJ.H'])
meta_data_concat['SEQUENCE_INPUT.L'] = meta_data_concat['SEQUENCE_INPUT.L'].fillna(meta_data_concat['SEQUENCE_VDJ.L'])

In [183]:
# concat_df['VH_nuc'] = meta_data_concat['SEQUENCE_INPUT.H']
# concat_df['VL_nuc'] = meta_data_concat['SEQUENCE_INPUT.L']

In [34]:
concat_df.index = concat_df.index.get_level_values(0) +':' + concat_df.index.get_level_values(1)

By default, the LIBRA-seq pipeline outputs nucleotide sequences. We align these to a reference genome and annotate using IMGT HighV-Quest (https://www.imgt.org/HighV-QUEST/home.action) for which you need fasta inputs

In [185]:
def write_to_fasta(input_csv, output_fasta_name): # 
    output = input_csv.copy()

    output.index = '>' + output.index

    hchains = pd.DataFrame(index=output.index)
    hchains['Seq'] = output['VH_nuc']
    hchains.index = hchains.index + ' HC'

    lchains = pd.DataFrame(index=output.index)
    lchains['Seq'] = output['VL_nuc']
    lchains.index = lchains.index + ' LC'

    output = pd.concat([hchains, lchains])


    path = output_fasta_name + '.fasta'

    with open(path, "w") as f:

        for i in range(len(output)):
            feature = i + 1
            header = output.iloc[i].name
            line1 = output.iloc[i][0]

            # break
            f.write(header + '\n')

            # print('line1 + '\n')
            f.write(line1 + '\n')

write_to_fasta(concat_df, 'published_LIBRA-seq_binders_genV2_24-03-06')

In [186]:
# concat_df.to_csv('published_LIBRA-seq_binders_genV2_24-03-06.csv')

In [187]:
imgt_aa_seqs = pd.read_csv('./published_LIBRA_seq_binders_genV2_24_03_06_IMGT/5_AA-sequences.txt', sep='\t') # Read output from IMGT

In [188]:
imgt_aa_seqs.head(1)

,Sequence number,Sequence ID,V-DOMAIN Functionality,V-GENE and allele,J-GENE and allele,D-GENE and allele,V-D-J-REGION,V-J-REGION,V-REGION,FR1-IMGT,CDR1-IMGT,FR2-IMGT,CDR2-IMGT,FR3-IMGT,CDR3-IMGT,JUNCTION,J-REGION,FR4-IMGT
0,1,1655331:AGTTGGTAGTCTTGCA_11_HC,productive,"Homsap IGHV4-4*07 F, or Homsap IGHV4-4*12 F",Homsap IGHJ3*02 F,Homsap IGHD2-15*01 F,QMQLQESGPGLVKSSETLSLTCSVSGDSIDGYYWSWIRQAADKRLE...,NaN,QMQLQESGPGLVKSSETLSLTCSVSGDSIDGYYWSWIRQAADKRLE...,QMQLQESGPGLVKSSETLSLTCSVS,GDSIDGYY,WSWIRQAADKRLEWIGR,IHTSGTT,NFNPSLRSRVTMSVDTSENQVSLRLTSVTAADTAVYYC,ARDLVVVLSAREVDDAFDI,CARDLVVVLSAREVDDAFDIW,DAFDIWGQGTTVTVSS,WGQGTTVTVSS


In [189]:
concat_df['VH_AA'] = concat_df.index + '_HC'
concat_df['VH_AA'] = concat_df['VH_AA'].map(dict(zip(imgt_aa_seqs['Sequence ID'], imgt_aa_seqs['V-D-J-REGION'])))

concat_df['VL_AA'] = concat_df.index + '_LC'
concat_df['VL_AA'] = concat_df['VL_AA'].map(dict(zip(imgt_aa_seqs['Sequence ID'], imgt_aa_seqs['V-J-REGION'])))

In [190]:
antigen_seqs = pd.read_csv('../antigen_sequences_LIBRA-seq_24-01-08.csv')
antigen_seqs = antigen_seqs[~antigen_seqs['Seq'].isna()]

In [191]:
# Removed antigens we have low confidence in (from followup analysis)
mutated_bg505_ants = ['HIV_BG505_D279K', 'HIV_BG505_D368R',
       'HIV_BG505_K169E', 'HIV_BG505_N160K', 'HIV_BG505_N332T',
       'HIV_BG505_DKO', 'HIV_BG505_T332N','BG505_6R_N332', 'CZA97_6R_N332', 'BG505sc_N332T', 'CZA97sc_N332T',
       'BG505sc_N160K', 'CZA97sc_N160K', 'BG505sc_K169E', 'CZA97sc_K169E',
       'BG505sc_D279K', 'CZA97sc_D279K', 'BG505sc_D368R', 'CZA97sc_D368R',
       'BG505sc_DKO', 'CZA97sc_DKO', 'CoV_MERS', 'MERS', 'CoV_MERSRBD']

In [192]:
concat_df = concat_df[~concat_df['Antigen'].isin(mutated_bg505_ants)]

In [193]:
concat_df.shape

(1616, 9)

In [194]:
concat_df['Antigen'].value_counts()[concat_df['Antigen'].value_counts() >= 5].index

Index(['CoV_SARS1', 'CoV_SARS2', 'HIV_ConC', 'HIV_A244', 'CoV_SARS2NTD',
       'CoV_SARS2D614G', 'CoV_OC43', 'HIV_ZM197', 'HIV_CZA97', 'HIV_CNE55',
       'FLU_HANC99', 'FLU_HA_NC99', 'HA_NC99', 'COV_MERS', 'COV_OC43',
       'CoV_HKU1', 'COV_SARS', 'COV_SARS2', 'SARS2_UK', 'COV_HKU1', 'CoV_HP6',
       'CoV_229E', 'CoV_NL63', 'SARS2_SA', 'FLU_HA_Michigan', 'SARS2',
       'RaTG13', 'COV_S1Fd', 'CoV_HP1'],
      dtype='object', name='Antigen')

In [196]:
concat_df[concat_df['Antigen'] == 'CoV_SARS2']['Vantage_ID'].value_counts()

Vantage_ID
5885    174
5404    127
5812     21
Name: count, dtype: int64

In [197]:
antigen_conversion = {'HIV_ZM197':'HIV_ZM197_Env', 'HIV_CZA97':'HIV_CZA97_Env', 'HIV_ConC':'ConC', 'HIV_CNE55': 'HIV_CNE55_Env', 'HIV_ZM197':'HIV_ZM197_Env', 
                      'FLU_HANC99':'HA_H1-NC99', 'FLU_HA_NC99':'HA_H1-NC99', 'HA_NC99':'HA_H1-NC99', 'FLU_HA_Michigan':'H1N1_A_Michigan_45-2015_HA',
                     'COV_OC43':'CoV_OC43', 'CoV_OC43':'CoV_OC43', 'CoV_SARS2D614G':'CoV_SARS2D614G', 'CoV_HP6':'SARS2_Hexa',
                     'CoV_SARS1': 'CoV_SARS1', 'COV_SARS':'CoV_SARS1', 'COV_SARS2':'SARS2_Hexa', 'CoV_SARS2':'SARS2_Hexa',
                     'CoV_SARS2NTD': 'SARS2_NTD', 'CoV_HKU1':'CoV_HKU1', 'COV_HKU1':'CoV_HKU1'
                     }

In [198]:
concat_df['antigen_name'] = concat_df['Antigen'].map(antigen_conversion)
concat_df['antigen_seq'] = concat_df['antigen_name'].map(dict(zip(antigen_seqs['Antigen'], antigen_seqs['Seq'])))

In [205]:
# concat_df.to_csv('published_LIBRA-seq_binders_genV2_24-03-09.csv')